In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [30]:
print_txt = False
print_compaire = True

In [3]:
riskDebtAccounts_files = [x for x in os.listdir() if 'risk' in x and '.xl' in x and '~' not in x]
riskDebtAccounts_files

['riskDebtAccounts_(16).xlsx']

In [4]:
data_riskDebtAccounts = pd.read_excel(riskDebtAccounts_files[0], header=5)
data_riskDebtAccounts.head(2)

,active,action,type,nameIFRS,codeIFRS,sourceSystem,accountCode,accountPlanType,reportType,bscCode,bsxCode,overdueFlag,ocpFlag,resetOverdueFlag
0,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в евро,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX303,NaN,Y,NaN
1,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в браз...,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX304,NaN,Y,NaN


In [5]:
data_riskDebtAccounts_work = data_riskDebtAccounts[:]
# xxmdg."riskDebtAccounts"."ocpFlag" = 'Y'
# xxmdg."riskDebtAccounts"."active" = 'Y'
# xxmdg."riskDebtAccounts"."action" = 'Include'
data_riskDebtAccounts_work = data_riskDebtAccounts_work[(data_riskDebtAccounts_work.ocpFlag=='Y')&(data_riskDebtAccounts_work.active=='Y')&(data_riskDebtAccounts_work.action=='Include')]
data_riskDebtAccounts_work = data_riskDebtAccounts_work[['type','sourceSystem','accountCode','reportType','bscCode']]
# pivots
pivot_1 = data_riskDebtAccounts_work.groupby(['bscCode','accountCode','reportType'],dropna=False,as_index=False)[['sourceSystem','type']].agg(lambda x: str(set(x)))
# pivot_1.loc[(~pivot_1.accountCode.isna()),'accountCode_like'] = pivot_1.loc[(~pivot_1.accountCode.isna())&(pivot_1.accountCode.str.contains('%')),'accountCode'] 
pivot_1['accountCode_like'] = pivot_1.accountCode.str.extract(r'(\d{2}%)') # col for like statements
pivot_2 = pivot_1.groupby(['sourceSystem','reportType', 'type'],dropna=False,as_index=False)[['bscCode','accountCode','accountCode_like']].agg(lambda x: str(set(x)))
pivot_2 = pivot_2.replace({'nan,':'', ', nan':'', '{':'', '}':'', '\"':'', 'nan':np.nan}, regex=True)
pivot_2

,sourceSystem,reportType,type,bscCode,accountCode,accountCode_like
0,"'ETL document loader', 'OEBS12_SA', 'OEBS12_EU'",AR,'Other receivables',"'BSC145', 'BSC140'",NaN,NaN
1,"'ETL document loader', 'OEBS12_SA', 'OEBS12_EU'",AR,'Trade receivables',"'BSC135', 'BSC130'",NaN,NaN
2,"'ETL document loader', 'SAP_HRG', 'OEBS12_EU',...",AP,'Other payables',"'BSC315', 'BSC310'",NaN,NaN
3,"'ETL document loader', 'SAP_HRG', 'OEBS12_EU',...",AP,'Provisions','BSC330',NaN,NaN
4,"'ETL document loader', 'SAP_HRG', 'OEBS12_EU',...",AP,'Trade payables',"'BSC300', 'BSC305'",NaN,NaN
5,'OEBS12_EU',AR,'Trade receivables','BSC130','015210',NaN
6,'SAP ECP',AP,'Other payables',NaN,"'5101010100', '5503030010', '5501010000', '550...",NaN
7,'SAP ECP',AP,'Provisions',NaN,"'2201910008', '2201910000'",NaN
8,'SAP ECP',AP,'Trade payables',NaN,"'5201010200', '5201010100', '5201011100', '520...",NaN
9,'SAP ECP',AR,'Other receivables',NaN,"'2602001000', '2601010009', '2602000010', '260...",NaN


In [7]:
pivot_2_SQL = pivot_2.copy()
# to sql statements
pivot_2_SQL.sourceSystem = pivot_2_SQL.sourceSystem.apply(lambda x: f'WHEN "sourceSystem" in ({x})')
pivot_2_SQL.reportType = pivot_2_SQL.reportType.apply(lambda x: f'and "reportType" = \'{x}\'')
pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'] = pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'].apply(lambda x: f'and "bscCode" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'] = pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'].apply(lambda x: f'and ("glAccount" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.accountCode_like.isna(), 'accountCode_like'] = pivot_2_SQL.loc[~pivot_2_SQL.accountCode_like.isna(), 'accountCode_like'].apply(lambda x: f'or "glAccount" like {x})')
pivot_2_SQL.loc[(pivot_2_SQL.accountCode_like.isna())&(~pivot_2_SQL.accountCode.isna()), 'accountCode_like'] = ')'
pivot_2_SQL.type = pivot_2_SQL.type.apply(lambda x: f'THEN {x}')
# pivot_2_SQL
# to SQL
pivot_2_SQL = pivot_2_SQL[['sourceSystem','reportType','bscCode','accountCode','accountCode_like','type']]
case_statement = pivot_2_SQL.to_string(header=False, index=False, na_rep='')
### QUERY
query = f"""CASE
            {case_statement}
            END AS "accountPurpose"
            from RISKACCESS."unifiedARReport" 
            where "rowType" = 'M' 
"""
# to text
if print_txt==True:
    with open(f'{str(date.today())}_SQL_query.txt', 'w') as file:
        file.write(query)

проверка старого запроса

In [8]:
old_query = f'''
WHEN "bscCode" in ('130', '135', 'BSC130', 'BSC135') 
THEN 'Trade receivables'
WHEN "bscCode" in ('160', '165', 'BSC160', 'BSC165') 
THEN 'Advances to suppliers'
WHEN "bscCode" in ('170', '175', 'BSC170', 'BSC175')
THEN 'Capital advances'
WHEN "bscCode" in ('140', '145', 'BSC140', 'BSC145') 
THEN 'Other receivables'
WHEN "bscCode" in ('330', 'BSC330')
THEN 'Provisions'
WHEN "bscCode" in ('300', '305', 'BSC300', 'BSC305') 
THEN 'Trade payables'
WHEN "bscCode" in ('310', '315', 'BSC310', 'BSC315') 
THEN 'Other payables'
WHEN "bscCode" in ('360', '365', 'BSC360', 'BSC365') 
THEN 'Advances received'
WHEN "bscCode" in ('350', '355', 'BSC350', 'BSC355') 
THEN 'Loans received'
WHEN "bscCode" in ('210', '215', 'BSC210', 'BSC215') 
THEN 'Loans issued'
WHEN "bscCode" in ('340', 'BSC340') 
THEN 'Tax'
WHEN "bscCode" in ('410', 'BSC410') 
THEN 'Pension'
ELSE 'Unknown'
"sourceSystem" in ('OEBS12_EU', 'ETL document loader', 'SAP_HRG') and
"rowType" = 'M' and
WHEN ("glAccount" like '62%' or "glAccount" in (7699200310, 7699100210, 4601000000, 4611000000, 1501010009, 1501010019, 1501010109, 1501010119, 2201010000, 2201010008, 2201010009, 2201010010, 2201010018, 2201010019, 2201011000, 2201011008, 2201011009, 2201011010, 2201011018, 2201011019, 2201010098, 2201010099, 2201020000, 2201020008, 2201030000, 2201030008, 2201010090, 2201010190, 2110000000, 2110000900, 2130000000, 2130000900, 2151000000, 2151000900, 2170000000, 2170000900, 1210000000, 1210000900, 1260000000, 2181000000, 2181000900))
THEN 'Trade receivables'
WHEN "glAccount" in (6002770900, 6002090900, 6002020000, 6002020099, 6002080000, 6002090000, 6002050000, 6002020900, 6002070000, 6002070099, 6002770000, 6002050900, 6002070900, 6099000215, 6099000224, 6099000212, 6099000217, 6099000219, 6099000619, 6099000120, 6099000123, 6099000820, 6099000430, 6099000460, 6099000470, 6099000490, 7699100410, 6099000830, 6099000810, 6002030000, 6002040000, 6002030900, 6002040900, 6099000440, 6099000450, 6099000203, 6099000615, 6002140000, 6002140099, 6099000616, 6002150000, 7610001900, 7610001000, 7610001099, 2301010100, 2301010108, 2301010110, 2301010118, 2301010200, 2301010208, 2301010210, 2301010218, 2301010300, 2301010308, 2301010400, 2301010408, 2301011100, 2301011108, 2301011110, 2301011118, 2301011200, 2301011208, 2301011210, 2301011218, 2301011300, 2301011308, 2302010000, 2302010007, 1611000000, 1613000000, 1615000000, 7610007900, 7610007000, 7610005000, 7699100620, 7699100420)
THEN 'Advances to suppliers'
WHEN "glAccount" in (6002010000, 6002010099, 6002010900, 6099000612, 6099000211, 2301020100, 2301020108, 2301020110, 2301020118, 2301020200, 2301020208, 2301020210, 2301020218, 2301020300, 2301020308, 2301020310, 2301020318, 2301021100, 2301021108, 2301021110, 2301021118, 2301021200, 2301021208, 2301021210, 2301021218, 2301021300, 2301021308, 2301021310, 2301021318, 2301000008, 2301001008, 2302020000, 2302020007, 1612000000, 1614000000)
THEN 'Capital advances'
WHEN "glAccount" in (6001089000, 7625003000, 7610003000, 7615003000, 7699100550, 6099000710, 7699200500, 7615004000, 7629009000, 7625013000, 7625013900, 7699100200, 7629000900, 7610406000, 7610406900, 7610405000, 7610405900, 7630707000, 7622020000, 7620801000, 7620801900, 7630705000, 7629000099, 7629000000, 7610004000, 7618000900, 7618000000, 6001080900, 6001080000, 7622010000, 7620703000, 7620704000, 7620701000, 7620701900, 7620702000, 7622030000, 7622040000, 7699900000, 7610500000, 7610500099, 7699100240, 7630702000, 7630703000, 7630706000, 7625006000, 7625007000, 7625001000, 7625001900, 7620200000, 7620200900, 2601030000, 2601030008, 2601040000, 2601040008, 2601050009, 2602000000, 2602000008, 2602000009, 2602000010, 2602000018, 2602000019, 2602001000, 2602001008, 2602001010, 2602001018, 2602010000, 2602010009, 2602060100, 2602000090, 2603000000, 2603000100, 36307060, 1284000000, 1284000900, 1630000000, 1282000000, 1282000900, 1251000000, 2601010009, 2601020009)
THEN 'Other receivables'
WHEN "glAccount" in (2201910000, 1290000000, 2190000000, 2301910100, 2602900000)
THEN 'Provisions'
WHEN ("glAccount" in (1022200000))
THEN 'FX Spot'
WHEN ("glAccount" in (1110009000, 1270000000, 1271000000, 2204000000, 2604010100, 2604020100, 2604021110, 2604021200, 7621156000))
THEN 'Loans issued'
WHEN ("glAccount" in (1410000000, 1430010000, 1430030000, 1430050000, 1430060000, 1430070000))
THEN 'Tax'
WHEN ("glAccount" in (2602000020))
THEN 'MTM Derivatives'        
WHEN ("glAccount" in (7610300000))
THEN 'Dividends'        
WHEN ("glAccount" in (7611102000))
THEN 'Capital expenses'       
ELSE 'Unknown'
"sourceSystem" in ('SAP ERP', 'SAP ECP') and
"rowType" = 'M' and
"reportType" = 'AR' and 
WHEN "glAccount" like '63%'
THEN 'Provisions'
WHEN ("glAccount" in (5201010100, 5201010110, 5201010200, 5201011100, 5201011110, 5201011200))
THEN 'Trade payables'
WHEN ("glAccount" in (5501000000, 5501010000, 5502000000, 5503020000, 5503030000, 5503030010, 5503010000))
THEN 'Other payables'
WHEN ("glAccount" like '62%' or "glAccount" in (5301010000, 5301010010, 7620002000, 0035100000))
THEN 'Advances received'
WHEN ("glAccount" like '60%' or "glAccount" in (5101010100, 5101020110, 5101021110, 5104040000, 5109000000, 5503050000, 5505010200, 5505010210, 5505011210, 5505020100, 7610101000, 7610402000, 7610404000, 7610405000, 7610601000, 7610603000, 7610691000, 7619000000, 7620002000, 0033970000, 0033500000, 0033600000, 0033960000, 0033140000, 0033130000, 0033120000, 0033110000, 0033800000, 0033950000, 0033970100))
THEN 'Other payables'
WHEN ("glAccount" in (0032200000, 0032310000))
THEN 'Pension'
WHEN ("glAccount" in (0032300000, 0032150000, 0032140000, 0032130000, 0032120000, 0032110000, 0031500000, 0033970200))
THEN 'Social'
WHEN ("glAccount" in (0031940000, 0031930000, 0031920000, 0031910000, 0031800000, 0031700000, 0031600000, 0031330000, 0031320000, 0031220000, 0031210000, 0031100000))
THEN 'Tax'
WHEN ("glAccount" in (0030200000))
THEN 'Loans received'
ELSE 'Unknown'
"sourceSystem" in ('SAP ERP', 'SAP ECP') and
"rowType" = 'M' and
"reportType" = 'AP' and 
'''

In [9]:
find_list = ['sourceSystem', 'THEN', 'reportType','WHEN']
str_df = pd.DataFrame({'string':old_query.split('\n')})
# dlete trash rows
str_df = str_df[(str_df.string.str.contains(find_list[0]))|(str_df.string.str.contains(find_list[1]))|(str_df.string.str.contains(find_list[2]))|(str_df.string.str.contains(find_list[3]))]
# add cols
for counter in range(0,len(find_list)-1):
    condition = find_list[counter]
    str_df[condition] = str_df.loc[str_df.string.str.contains(condition), 'string']
    str_df[condition] = str_df[condition].bfill()
for counter in ['bscCode', 'glAccount', 'like']:
    str_df[counter] = str_df.loc[str_df.string.str.contains(counter), 'string']
# dropna
str_df = str_df[(~str_df.bscCode.isna())|(~str_df.glAccount.isna())]
## formatting 
# Type col
str_df.THEN = str_df.THEN.str[5:]
type_list = pivot_2.type.unique().tolist()
str_df = str_df[str_df.THEN.isin(type_list)]
# reportRtpe col
str_df.reportType = str_df.reportType.str.extract(r'(AP|AR)')
# cols with brackets
expression_brackets = r'in \(([^)]*)\).*'
cols_brackets_list = ['sourceSystem','bscCode','glAccount']
for col in cols_brackets_list:
    str_df[col] = str_df[col].str.extract(expression_brackets)
# like col
str_df.like = str_df.like.str.extract(r'(\'\d{2}%\')')
# drop col 
str_df = str_df.drop(columns='string')
str_df = str_df.rename(columns={'sourceSystem':'sourceSystem_old','THEN':'type_old','reportType':'reportType_old','bscCode':'bscCode_old','glAccount':'accountCode_old','like':'accountCode_like_old'})
str_df = str_df[['sourceSystem_old','reportType_old','type_old','bscCode_old','accountCode_old','accountCode_like_old']]
str_df.head(2)

,sourceSystem_old,reportType_old,type_old,bscCode_old,accountCode_old,accountCode_like_old
1,"'OEBS12_EU', 'ETL document loader', 'SAP_HRG'",AR,'Trade receivables',"'130', '135', 'BSC130', 'BSC135'",NaN,NaN
7,"'OEBS12_EU', 'ETL document loader', 'SAP_HRG'",AR,'Other receivables',"'140', '145', 'BSC140', 'BSC145'",NaN,NaN


In [11]:
# Creating list of accountCode in both bases
ls = pivot_2.accountCode.replace({'\"':'','\'':''}).str.split(',').tolist()
res = []
for item in ls:
    try:
        for item_i in item:
            item_i = item_i.strip()
            item_i = item_i.strip('\'')
            # item_i = int(item_i)
            res.append(item_i)
    except:
        continue
ls = str_df.accountCode_old.str.split(',').to_list()
res_old = []
for item in ls:
    try:
        for item_i in item:
            item_i = item_i.strip()
            item_i = item_i.strip('\'')
            # item_i = int(item_i)
            res_old.append(item_i)
    except:
        continue
# creating list to SQL query
have_only_old = [item for item in res_old if item not in res]
have_only_old_str = str(have_only_old)
have_only_old_str = have_only_old_str[1:-1]
have_only_new = [item for item in res if item not in res_old]
have_only_new_str = str(have_only_new)
have_only_new_str = have_only_new_str[1:-1]

['015210',
 '5101010100',
 '5503030010',
 '5501010000',
 '5503030000',
 '5501000000',
 '5503020000',
 '5505010210',
 '5505020100',
 '5502000000',
 '5503010000',
 '5104040000',
 '2201910008',
 '2201910000',
 '5201010200',
 '5201010100',
 '5201011100',
 '5201011110',
 '5201011200',
 '5201010110',
 '2602001000',
 '2601010009',
 '2602000010',
 '2601030008',
 '2601040008',
 '2602000009',
 '2602010009',
 '2603000000',
 '2601040000',
 '2602001008',
 '2602000008',
 '2601020009',
 '2602000019',
 '2603000100',
 '2602060100',
 '2601050009',
 '2602010000',
 '2602000090',
 '2602000000',
 '2601030000',
 '2602000018',
 '2602001018',
 '2602001010',
 '2201010190',
 '2201011019',
 '2201010010',
 '2201010009',
 '2201011009',
 '220199XX90',
 '1501010109',
 '2201010008',
 '2201010018',
 '2201020008',
 '2201030008',
 '1501010119',
 '2201030000',
 '2201010000',
 '2201011000',
 '2201011008',
 '220109XX10',
 '2201011018',
 '2201011010',
 '2201010019',
 '1501010019',
 '2201010098',
 '2201010099',
 '2201010090',

In [22]:
query = f'''
select distinct "customerName","orgName", "companyCode","codeBUsap", "dueDate", "presentAmountUSD","holding","currencyCode","reportType","glAccount","sourceSystem", "reportDate"
from RISKACCESS."unifiedARReport" 
where "rowType" = 'M' and 
"sourceSystem" in ('SAP ERP', 'SAP ECP')
--"reportType" = 'AR' 
and "glAccount" = '5101020110'
--in ({have_only_old_str}) 
and "presentAmountUSD" is not null
fetch first 10 rows only'''
data_1 = export_from_RISKCUSTOM(query)
data_1['comment'] = 'Нет в новом запросе'
data_1.head(2)

,customerName,orgName,companyCode,codeBUsap,dueDate,presentAmountUSD,holding,currencyCode,reportType,glAccount,sourceSystem,reportDate,comment
0,Corrigo Fertilizers FZ-LLC,Eurochem Trading ME DMCC,AETRD,E920,2024-01-19,-7499887.69,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-02-29,Нет в новом запросе
1,Corrigo Fertilizers FZ-LLC,Eurochem Trading ME DMCC,AETRD,E920,2024-02-29,22999655.58,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-02-29,Нет в новом запросе
2,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-05-21,-600000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-07-31,Нет в новом запросе
3,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-05-30,4600000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
4,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-03-12,-7000000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
5,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-05-01,5000000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
6,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-03-05,-5400000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
7,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-04-24,-4000000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
8,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-03-05,-15000000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе
9,EuroChem Trading Middle East FZE,Corrigo Fertilizers LLC,AECOR,E910,2024-04-16,-10000000.00,EUROCHEM,USD,AP,5101020110,SAP ECP,2024-05-31,Нет в новом запросе


In [16]:
query = f'''
select distinct "customerName","orgName", "companyCode","codeBUsap", "dueDate", "presentAmountUSD","holding","currencyCode","reportType","glAccount","sourceSystem", "reportDate"
from RISKACCESS."unifiedARReport" 
where "rowType" = 'M' 
--and 
--"sourceSystem" in ('SAP ERP', 'SAP ECP') and
--"reportType" = 'AR' 
and "glAccount" in ({have_only_new_str}) 
and "presentAmountUSD" is not null
fetch first 10 rows only'''
data_2 = export_from_RISKCUSTOM(query)
data_2['comment'] = 'Нет в старом запросе'
data_2.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,customerName,orgName,companyCode,codeBUsap,dueDate,presentAmountUSD,holding,currencyCode,reportType,glAccount,sourceSystem,reportDate,comment
0,CZAV U.A.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-11-10,-98.67,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
1,IANNACCONE ANTONIO S.R.L.,ITEAG: EuroChem Agro Spa,ITEAG,None,2022-01-31,-167.01,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
2,AGROCHIMICA DISTRIBUZIONE SRL,ITEAG: EuroChem Agro Spa,ITEAG,None,2022-03-03,601.69,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
3,TRIFERTO B.V.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-10-10,-2338.14,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
4,CZAV U.A.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-08-10,-96.67,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
5,RAIFFEISEN-WAREN-ZENTRALE RHEIN-MAIN E.G.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-09-10,-198.27,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
6,BEISELEN GMBH,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-12-10,-85.64,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
7,C.A.P.A.C.SOC.COOP.AGRICOLA,ITEAG: EuroChem Agro Spa,ITEAG,None,2022-04-30,-178.14,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
8,RAIFFEISEN-WAREN-ZENTRALE RHEIN-MAIN E.G.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2022-02-10,-195.47,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе
9,RAIFFEISEN-WAREN-ZENTRALE RHEIN-MAIN E.G.,DEEAG: EuroChem Agro GmbH,DEEAG,None,2021-03-10,-188.61,EUROCHEM,EUR,AR,015210,OEBS12_EU,2022-01-31,Нет в старом запросе


In [23]:
# manual
data_1['type'] = 'Other payables'
data_2['type'] = 'Trade receivables'

In [33]:
if print_compaire==True:
    with pd.ExcelWriter(f'{str(date.today())}_data_compaire.xlsx', 'openpyxl') as writer:
        pd.concat([data_1,data_2]).to_excel(writer, sheet_name='data', index=False)
        pd.concat([pivot_2, str_df], axis=0).to_excel(writer, sheet_name='query', index=False)
        pd.DataFrame({'have_only_new':have_only_new}).to_excel(writer, sheet_name='compaire', index=False, startcol=0)
        pd.DataFrame({'have_only_old':have_only_old}).to_excel(writer, sheet_name='compaire', index=False, startcol=2)
    writer.close()

ValueError: I/O operation on closed file